In [1]:
!nvidia-smi

Sun Oct 16 23:51:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   35C    P8    17W / 170W |   3244MiB / 12288MiB |     28%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda import gpuarray
from skcuda import linalg

/home/lum1narie/.local/lib/python3.10/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [3]:
REP_N = 1000
ARRAY_SIZE = 1000

linalg.init()

A = gpuarray.to_gpu(np.array([[i + j for j in range(ARRAY_SIZE)] for i in range(ARRAY_SIZE)], np.float32))
x = gpuarray.to_gpu(np.array([[i - j for j in range(ARRAY_SIZE)] for i in range(REP_N)], np.float32))
y = gpuarray.empty((REP_N, ARRAY_SIZE), np.float32)

In [4]:
%%time
for i in range(REP_N):
    y[i] = linalg.dot(A, x[i])

CPU times: user 86.5 ms, sys: 3.62 ms, total: 90.1 ms
Wall time: 89.1 ms
